### **남은 Task : GameRulesetName 변수 파악, agent1, agent2 변수 파악 등**

## UM - Game-playing Strength of MCTS variants

### **1. Imports**

---

In [5]:
import pandas as pd
import numpy as np
import torch

### **2. Data**

---



In [6]:
df_train = pd.read_csv("/root/kag_comp/UM_MCTS/Data/train.csv")
df_test = pd.read_csv("/root/kag_comp/UM_MCTS/Data/test.csv")
df_concepts = pd.read_csv("/root/kag_comp/UM_MCTS/Data/concepts.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

`-` 컬럼 정보 확인

In [4]:
print(f"DF shape : {df_train.shape}")
print(f"Columns : {df_train.columns}")
print(f"Rules count : {df_train.EnglishRules.nunique()}") ## Natural Language
print(f"Lud Rule count : {df_train.LudRules.nunique()}")  ## Ludii에서의 게임 언어로 설명

DF shape : (233234, 814)
Columns : Index(['Id', 'GameRulesetName', 'agent1', 'agent2', 'Properties', 'Format',
       'Time', 'Discrete', 'Realtime', 'Turns',
       ...
       'DoLudeme', 'Trigger', 'PlayoutsPerSecond', 'MovesPerSecond',
       'EnglishRules', 'LudRules', 'num_wins_agent1', 'num_draws_agent1',
       'num_losses_agent1', 'utility_agent1'],
      dtype='object', length=814)
Rules count : 1328
Lud Rule count : 1373


In [5]:
df_train.dtypes

Id                     int64
GameRulesetName       object
agent1                object
agent2                object
Properties             int64
                      ...   
LudRules              object
num_wins_agent1        int64
num_draws_agent1       int64
num_losses_agent1      int64
utility_agent1       float64
Length: 814, dtype: object

In [6]:
nlst = []

for i in range(4, 806) :
    nlst.append(df_train.iloc[:,i].nunique())

`-` Descriptions of the each columns

In [7]:
print(nlst)

[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 127, 35, 31, 32, 225, 171, 149, 217, 6, 56, 82, 4, 181, 110, 132, 56, 26, 26, 21, 21, 11, 26, 18, 5, 2, 4, 146, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 26, 21, 8, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 77, 47, 84, 77, 47, 84, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 56, 2, 3, 2, 2, 49, 2, 7, 2, 31, 2, 2, 2, 2, 91, 2, 90, 2, 10, 2, 18, 2, 74, 2, 76, 2, 15, 2, 3, 2, 27, 2, 30, 1, 1, 2, 8, 2, 54, 2, 52, 2, 44, 2, 20, 1, 1, 1, 3, 2, 4, 2, 5, 2, 6, 1, 87, 1, 68, 2, 97, 2, 30, 2, 45, 2, 12, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 44, 2, 3, 2, 48, 2, 45, 2, 2, 43, 2, 18, 2, 16, 2, 2, 2, 2, 2, 2, 12, 2, 67, 2, 4, 2, 4, 2, 2, 89, 2, 89, 2, 16, 2, 4, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 

In [8]:
print(df_concepts.nunique())
display(df_concepts.head(10))

Id                   809
Name                 809
Description          793
TypeId                12
DataTypeId             3
ComputationTypeId      2
TaxonomyString       809
LeafNode               2
ShowOnWebsite          2
dtype: int64


,Id,Name,Description,TypeId,DataTypeId,ComputationTypeId,TaxonomyString,LeafNode,ShowOnWebsite
0,1,Properties,General properties of the game.,1,1,1,1,0,0
1,2,Format,Format of the game.,1,1,1,1.1,0,0
2,3,Time,Time model.,1,1,1,1.1.1,0,0
3,4,Discrete,Players move at discrete intervals.,1,1,1,1.1.1.1,1,1
4,5,Realtime,Moves not discrete.,1,1,1,1.1.1.2,1,1
5,6,Turns,Player turns.,1,1,1,1.1.2,0,0
6,7,Alternating,Players take turns moving.,1,1,1,1.1.2.1,1,1
7,8,Simultaneous,Players can move at the same time.,1,1,1,1.1.2.2,1,1
8,9,Stochastic,Game involves chance elements.,1,1,1,1.1.3,1,1
9,10,HiddenInformation,Game involves hidden information.,1,1,1,1.1.4,1,1


> 리프 노드에 해당하는 열만 유의미할 것으로 판단됨. 다른 열 정보는 필요 없음(아마도)
>
> 단, TypeId 별로 변수들을 구분하는 것은 필요할듯.

* 리프노드라고 해도 값이 하나밖에 없는 경우도 있으며, 리프노드가 아니라고 해도 값이 여러개인 경우도 있음.

In [9]:
# ## LeafNode의 수 : 568
# list(np.array(df_concepts.Name[df_concepts.LeafNode == 1]))[:564]

# ## 나머지 : 241
# list(np.array(df_concepts.Name[df_concepts.LeafNode == 0]))[:-1]

In [10]:
## 모든 게임의 플레이어는 2명
df_train.TwoPlayer.nunique()

1

In [11]:
df_train.loc[:, list(np.array(df_concepts.Name[df_concepts.LeafNode == 1]))[:564]].nunique()[0:20]
# df_train.loc[:, list(np.array(df_concepts.Name[df_concepts.LeafNode == 0]))[:-1]].nunique()[0:20]

Discrete                1
Realtime                1
Alternating             1
Simultaneous            1
Stochastic              2
HiddenInformation       1
Match                   1
AsymmetricPlayRules     1
AsymmetricEndRules      1
AsymmetricSetup         1
AsymmetricPiecesType    2
Simulation              1
Solitaire               1
TwoPlayer               1
Multiplayer             1
Team                    2
Coalition               1
DeductionPuzzle         1
PlanningPuzzle          1
SquareShape             2
dtype: int64

In [12]:
df_train.loc[:, list(np.array(df_concepts.Name[df_concepts.LeafNode == 1]))[:564]].nunique()
df_train.loc[:, list(np.array(df_concepts.Name[df_concepts.LeafNode == 0]))[:-1]].nunique()

Properties              1
Format                  1
Time                    1
Turns                   1
Asymmetric              2
                       ..
InternalCounter         2
SetHidden               1
Efficiency              2
AlquerqueBoard          2
ThreeMensMorrisBoard    2
Length: 240, dtype: int64

In [13]:
df_train.loc[:, list(np.array(df_concepts.Name[df_concepts.LeafNode == 0]))[:-1]].Asymmetric.unique()

array([0, 1])

In [14]:
df_concepts.loc[(df_concepts.Name == 'AsymmetricForces'),:]

,Id,Name,Description,TypeId,DataTypeId,ComputationTypeId,TaxonomyString,LeafNode,ShowOnWebsite
15,16,AsymmetricForces,Players have different forces.,1,1,1,1.1.6.2,0,1


In [15]:
df_train.loc[:, list(np.array(df_concepts.Name[df_concepts.LeafNode == 0]))[:-1]].nunique().map(lambda x : True if x > 2 else False).sum()

0

> 리프 노드만 유의미한 값을 가지는 것은 아닌가봄, 단, 리프 노드가 아닌 것에서는 최소한 값이 바이너리를 넘어가지 않았음.

In [16]:
df_train.iloc[:, -4:]

,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,4,0,11,-0.466667
1,5,0,10,-0.333333
2,7,0,8,-0.066667
3,5,0,10,-0.333333
4,5,0,10,-0.333333
...,...,...,...,...
233229,2,0,13,-0.733333
233230,9,1,5,0.266667
233231,11,3,1,0.666667
233232,24,2,4,0.666667


> 그리고 마지막 3열은 승리와 패배에 대한 것임. 즉, 이들의 선형 결합으로 타겟이 구성됨.

`-` Test data 분석

* 훈련 데이터가 테스트 데이터의 범위를 포함하는지 여부를 확인

In [21]:
df_test ## 뭐임, 애초에 세개밖에 없네?

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules
0,233234,00Y,MCTS-UCB1-0.6-NST-false,MCTS-ProgressiveHistory-0.1-MAST-true,1,1,1,1,0,1,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
1,233235,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
2,233236,00Y,MCTS-UCB1Tuned-0.1-Random200-true,MCTS-ProgressiveHistory-0.1-MAST-true,1,1,1,1,0,1,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."


In [25]:
tmp = pd.concat([pd.DataFrame(df_train.nunique()), pd.DataFrame(df_test.nunique())], axis = 1)
tmp.columns = ['train', 'test']

In [76]:
tmp.index[(tmp.assign(minus = lambda _df : _df.train - _df.test) != 0.0).minus]

Index(['Id', 'GameRulesetName', 'agent1', 'agent2', 'Stochastic', 'Asymmetric',
       'AsymmetricForces', 'AsymmetricPiecesType', 'PlayersWithDirections',
       'Cooperation',
       ...
       'DoLudeme', 'Trigger', 'PlayoutsPerSecond', 'MovesPerSecond',
       'EnglishRules', 'LudRules', 'num_wins_agent1', 'num_draws_agent1',
       'num_losses_agent1', 'utility_agent1'],
      dtype='object', length=598)

In [85]:
# [set(df_test[i].unique()) - set(df_train[i].unique()) for i in df_test]

> 도메인이 다른가 확인했는데, 다르지 않음. 일단 테스트 데이터는 학습 데이터와 도메인이 같다고 가정할 수 있을듯. 추가적인 처리 필요 없음.

`-` 쓸모없는 행들(결측치만 존재)

In [64]:
set(df_train.columns[df_train.isna().sum() != 0])

{'Behaviour',
 'BoardCoverage',
 'BoardSitesOccupied',
 'BranchingFactor',
 'Clarity',
 'Complexity',
 'DecisionFactor',
 'Decisiveness',
 'Drama',
 'Duration',
 'GameOutcome',
 'MoveDistance',
 'MoveEvaluation',
 'PieceNumber',
 'ScoreDifference',
 'StateEvaluation',
 'StateEvaluationDifference',
 'StateRepetition'}

In [63]:
set(df_test.columns[df_test.isna().sum() != 0]) - set(df_train.columns[df_train.isna().sum() != 0])

set()

In [61]:
set(df_train.columns[df_train.isna().sum() != 0]) - set(df_test.columns[df_test.isna().sum() != 0])

set()

### **3. 무지성 catboost**

---

In [26]:
import catboost

In [30]:
df_train.columns

Index(['Id', 'GameRulesetName', 'agent1', 'agent2', 'Properties', 'Format',
       'Time', 'Discrete', 'Realtime', 'Turns',
       ...
       'DoLudeme', 'Trigger', 'PlayoutsPerSecond', 'MovesPerSecond',
       'EnglishRules', 'LudRules', 'num_wins_agent1', 'num_draws_agent1',
       'num_losses_agent1', 'utility_agent1'],
      dtype='object', length=814)

In [56]:
df_train.utility_agent1.unique()

array([-0.46666667, -0.33333333, -0.06666667,  0.06666667,  0.2       ,
       -0.2       , -0.73333333,  0.6       ,  0.46666667,  0.33333333,
       -0.6       ,  0.4       ,  0.13333333,  0.26666667,  0.53333333,
        0.73333333,  1.        ,  0.8       ,  0.93333333,  0.86666667,
       -0.13333333, -1.        ,  0.66666667, -0.26666667, -0.53333333,
        0.        , -0.4       ,  0.36666667, -0.86666667, -0.93333333,
       -0.66666667, -0.8       ,  0.28888889,  0.03333333, -0.96666667,
        0.56666667,  0.9       , -0.83333333, -0.43333333, -0.16666667,
        0.1       ,  0.3       , -0.5       , -0.56666667,  0.76666667,
       -0.7       ,  0.23333333, -0.76666667,  0.16666667, -0.1       ,
       -0.23333333,  0.96666667,  0.83333333, -0.63333333, -0.03333333,
        0.95555556, -0.95555556, -0.36666667, -0.3       ,  0.5       ,
        0.7       ,  0.63333333,  0.43333333, -0.9       ,  0.02222222,
        0.24444444,  0.44444444, -0.44444444])

In [58]:
## 하이퍼볼릭 탄젠트 시그모이드를 사용하는 것이 좋을듯. 일단 대충 끼워넣어보자.
df_train.utility_agent1.map(lambda x : 1/2*np.log((1+x)/(1-x)) if (x > -0.99999) & (x < 0.99999) else 1/2*np.log(0.00000001/2) if x > -0.99999 else 1/2*np.log(1.99999999/0.00000001)) ## 대충 -10에서 10정도로 변환

-9.556913962256155

In [86]:
df_train.utility_agent1.map(lambda x : 1/2*np.log((1+x)/(1-x)) if (x > -0.99999) & (x < 0.99999) else 1/2*np.log(0.00000001/2) if x > -0.99999 else 1/2*np.log(1.99999999/0.00000001))\
.map(lambda x : (np.exp(2*x) - 1)/(np.exp(2*x)+1))

0.99999999

In [166]:
X = df_train.drop(["num_wins_agent1", "num_draws_agent1", "num_losses_agent1","utility_agent1"], axis = 1)
y = df_train.utility_agent1.map(lambda x : 1/2*np.log((1+x)/(1-x)) if (x > -0.99999) & (x < 0.99999) else 1/2*np.log(0.00000001/2) if x > -0.99999 else 1/2*np.log(1.99999999/0.00000001))

predictr = catboost.CatBoostRegressor(cat_features = ["GameRulesetName", "agent1", "agent2", "EnglishRules", "LudRules"])
predictr.fit(X, y)

In [91]:
np.mean(abs(df_train.utility_agent1 - [(np.exp(2*x) - 1)/(np.exp(2*x)+1) for x in predictr.predict(X)]))

0.4591380819549907

In [92]:
import polars as pl

In [ ]:
def predict(test, submission):  
    test_transformed = enc.transform(test[obj_cols])
    for e, c in enumerate(obj_cols):
        test = test.with_columns(pl.Series(c, test_transformed[:, e]))
    return submission.with_columns(pl.Series(target, rf.predict(test)))

### **4. 무지성 제출**

In [ ]:
pl.read_csv()

In [135]:
df_train.shape

(233234, 814)

In [134]:
df_test.shape

(3, 810)

In [147]:
df_train.head()

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333


In [145]:
set(df_train.columns) - set(df_test.columns)

{'num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1', 'utility_agent1'}

In [129]:
df_test = pd.read_csv("/root/kag_comp/UM_MCTS/Data/test.csv")

In [ ]:
import os
import polars as pl
import numpy as np
import pandas as pd
import catboost
import kaggle_evaluation.mcts_inference_server

target = 'utility_agent1'
counter = 0  # 첫 번째 호출 확인용 변수

def train_model():
    global predictr
    df_train = pd.read_csv("/root/kag_comp/UM_MCTS/Data/train.csv")
    
    X = df_train.drop(["num_wins_agent1", "num_draws_agent1", "num_losses_agent1", target], axis = 1)
    y = df_train[target]

    predictr = catboost.CatBoostRegressor(cat_features=["GameRulesetName", "agent1", "agent2", "EnglishRules", "LudRules"], verbose = 0)
    predictr.fit(X, y)

def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    global counter
    if counter == 0:  # 첫 번째 호출 시에만 모델을 학습하거나 로드
        train_model()
        counter += 1
    
    predictions = predictr.predict(test.to_pandas())
    return sample_sub.with_columns(pl.Series('utility_agent1', predictions))

# 인퍼런스 서버 설정
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/root/kag_comp/UM_MCTS/Data/test.csv',
            '/root/kag_comp/UM_MCTS/Data/sample_submission.csv'
        )
)

`-` for model submission

In [15]:
%pip install grpcio
%pip install polars
%pip install google-cloud
%pip install google-cloud-vision
import kaggle_evaluation.mcts_inference_server

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **6. 자연어, 약속어**

---

`-` 무지성 토큰화 이후 변수 추가

In [2]:
## 문서를 벡터 표현으로 인스턴스화
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import string
from nltk.corpus import stopwords

# import nltk
# nltk.download('stopwords')

* 일단 자연어는 `EnglishRules`, `LudRules`이거 두개임. 하지만 `LudRules`은 약간 정련된 단어들로 이뤄져 있으므로 조금 분석이 더 필요함. 따라서 `EnglishRules`만 먼저 넣어보자.

> 라곤 했지만 그냥 무지성으로 때려박아보자!

`-` 전처리 과정 : 함수화 또는 파이프라인 필요

In [3]:
nl = df_train[['EnglishRules','LudRules']]

NameError: name 'df_train' is not defined

In [97]:
display(nl.EnglishRules.describe())
display(nl.LudRules.describe())

count                 233234
unique                  1328
top       Hares start first.
freq                    3874
Name: EnglishRules, dtype: object

count                                                233234
unique                                                 1373
top       (game "Ludus Coriovalli" (players 2) (equipmen...
freq                                                    370
Name: LudRules, dtype: object

In [223]:
## 구두점 제거 함수
def text_process(mess) :
    STOPWORDS = stopwords.words('english')
    nopunc = [ch for ch in mess if ch not in string.punctuation]
    nopunc = ''.join(nopunc)

    return ' '.join([w for w in nopunc.split() if w.lower() not in STOPWORDS])

## 구두점 제거 작업
nl_list = ["EnglishRules", "LudRules"]

for i in nl_list :
    nl[i] = nl[i].apply(text_process)

## 벡터라이징
vect_list = [CountVectorizer() for i in range(len(nl_list))]
nl_vect_list = []

for i, vect in enumerate(vect_list) :
    nl_vect_list.append(vect.fit_transform(nl[nl_list[i]]))

## TF-IDF로 변환
tfidf_list = [TfidfTransformer() for i in range(len(nl_list))]
nl_tfidf_list = []

for i, tfidf in enumerate(tfidf_list) :
    nl_tfidf_list.append(tfidf.fit_transform(nl_vect_list[i]).toarray())

* 정리

In [172]:
df_test = pl.read_csv("/root/kag_comp/UM_MCTS/Data/test.csv")

In [222]:
nl_list = ["EnglishRules", "LudRules"]
nl_test = df_test[nl_list]

## 구두점 제거
for i, l in enumerate(nl_list) :
    nl_test = nl_test.drop(l).insert_column(i, nl_test[l].map_elements(text_process))

processed_nl_test = []

for i, l in enumerate(nl_list) :
    processed_nl_test.append(tfidf_list[i].transform(vect_list[i].transform(nl_test[l])).toarray())

tfid_nl_test = pl.DataFrame(np.concatenate(processed_nl_test, axis = 1))

XX = pl.concat([df_test.drop(nl_list), tfid_nl_test], how = 'horizontal')

/tmp/ipykernel_899814/798260045.py:6: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  nl_test = nl_test.drop(l).insert_column(i, nl_test[l].map_elements(text_process))


`-` 그래서 이거 어떻게 분석함? 그냥 각 열 사용해서 쓰면 되는거임?(그런듯)

In [151]:
import polars as pl

In [ ]:
pl_train = pl.DataFrame(df_train)

In [161]:
tfid_nl = pl.DataFrame(np.concatenate([tfid_eng.toarray(), tfid_lud.toarray()], axis = 1))

In [170]:
target = 'utility_agent1'

X = pl.concat([pl_train.drop(["EnglishRules", "LudRules"]), tfid_nl], how = 'horizontal').drop(target)
y = pl_train[target]

In [145]:
df_train.drop(["EnglishRules", "LudRules"], axis = 1).
np.concatenate([tfid_eng.toarray(), tfid_lud.toarray()], axis = 1)

(233234, 3841)

In [150]:
np.concatenate([tfid_eng.toarray(), tfid_lud.toarray()], axis = 1).shape

(233234, 8633)

**모형 돌려보자!**

In [249]:
X = pd.DataFrame(X)
y = pd.Series(y)

In [257]:
predictr = catboost.CatBoostRegressor(cat_features = [1, 2, 3], verbose = 0)
predictr.fit(X, y)

In [1]:
yhat = predictr.predict(X)

NameError: name 'predictr' is not defined

In [ ]:
X = df_train.drop(["num_wins_agent1", "num_draws_agent1", "num_losses_agent1","utility_agent1"], axis = 1)
y = df_train.utility_agent1.map(lambda x : 1/2*np.log((1+x)/(1-x)) if (x > -0.99999) & (x < 0.99999) else 1/2*np.log(0.00000001/2) if x > -0.99999 else 1/2*np.log(1.99999999/0.00000001))

predictr = catboost.CatBoostRegressor(cat_features = ["GameRulesetName", "agent1", "agent2", "EnglishRules", "LudRules"])
predictr.fit(X, y)

np.mean(abs(df_train.utility_agent1 - [(np.exp(2*x) - 1)/(np.exp(2*x)+1) for x in predictr.predict(X)]))

In [ ]:
import os
import polars as pl
import numpy as np
import pandas as pd
import catboost
import kaggle_evaluation.mcts_inference_server

target = 'utility_agent1'
counter = 0  # 첫 번째 호출 확인용 변수

def train_model():
    global predictr
    df_train = pd.read_csv("/root/kag_comp/UM_MCTS/Data/train.csv")
    
    X = df_train.drop(["num_wins_agent1", "num_draws_agent1", "num_losses_agent1", target], axis = 1)
    y = df_train[target]

    predictr = catboost.CatBoostRegressor(cat_features=["GameRulesetName", "agent1", "agent2", "EnglishRules", "LudRules"], verbose = 0)
    predictr.fit(X, y)

def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    global counter
    if counter == 0:  # 첫 번째 호출 시에만 모델을 학습하거나 로드
        train_model()
        counter += 1
    
    predictions = predictr.predict(test.to_pandas())
    return sample_sub.with_columns(pl.Series('utility_agent1', predictions))

# 인퍼런스 서버 설정
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/root/kag_comp/UM_MCTS/Data/test.csv',
            '/root/kag_comp/UM_MCTS/Data/sample_submission.csv'
        )
)